importing libraries and downloading the dataset and converting it into a dataframe

In [39]:
import pandas as pd

import requests

from bs4 import BeautifulSoup
data = pd.read_csv("Geospatial_Coordinates.csv") 

req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df1 = pd.read_html(str(table))

df=pd.DataFrame(df1[0])


removing cells with 'Not assigned' values in Borough column

In [40]:
df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)

splitting cells using ','

In [41]:
df['Neighborhood'] = df['Neighborhood'].str.split(pat='/')
df['Neighborhood'] = df['Neighborhood'].apply(','.join)
df.loc[df['Borough'] == 'Not assigned', 'Borough'] = 10

replacing 'Not assigned' neighborhood with the value of Borough	

In [42]:
# Create a list of indices of the cells where the value of Column2 is "Not assigned"
idx_to_change = df.loc[df['Neighborhood'] == "Not assigned"].index

# Iterate the list of indicies and set Column2 to the value of Column1 at the given index
for i in idx_to_change:
    df.loc[i, 2] = df.loc[1,1]

In [43]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


adding the longitude and latitude 

In [54]:
merged_data = pd.merge(left=df, right=data, left_on='Postal code', right_on='Postal Code')
merged_data.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",M5A,43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",M6A,43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",M7A,43.662301,-79.389494


cleaning the final table

In [55]:
merged_data.drop(merged_data.columns[[3]], axis = 1, inplace = True) 
merged_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494
